# References

- https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
- https://github.com/fastai/course-v3/blob/master/nbs/dl1/lesson1-pets.ipynb
- https://www.kaggle.com/grassknoted/asl-alphabet/home

# Setup

Import libraries.

In [17]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy

Load the pre-trained model (ResNet34).

In [3]:
resnet34 = models.resnet34(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.torch/models/resnet34-333f7ec4.pth
46.1%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

82.6%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%


Replace the last fully-connected layer `fc`, which normally has 512 input features and 1000 output features.

In [22]:
# 512 for ResNet34
number_of_features = resnet34.fc.in_features

# letters A-Z plus SPACE, DELETE, and NOTHING
number_of_classes = 29

# freeze training for all layers
for param in resnet34.parameters():
    param.require_grad = False

# newly created modules have require_grad=True by default
resnet34.fc = nn.Linear(number_of_features, number_of_classes)
print(resnet34.fc)

Linear(in_features=512, out_features=29, bias=True)


Small script to help reformat the `val` folder that comes with this particular dataset.

In [ ]:
import os
import shutil

for filename in os.listdir('.'):
	# files are named: `A_test.jpg`, etc.
	if filename.endswith('.jpg'):
		
		# A, B, C, etc.
		folder_name = filename.split('_')[0]

		# make the new folder and move
		os.mkdir(folder_name)
		shutil.move(filename, '{}/{}'.format(folder_name, filename))

Load data.

In [24]:
data_dir = '/datasets/asl_alphabet/'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [35]:
print(class_names)
print(image_datasets['train'][0][0].size())

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
torch.Size([3, 224, 224])


# Training

Remap and normalize images, based on original ResNet34 training data.

In [12]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('using device:', device)

resnet34 = resnet34.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet34.parameters(), lr=0.001, momentum=0.9)
learning_rate_decay = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

using device: cuda:0
